In [10]:
import pickle
import os 
import itertools
import glob

import pandas as pd
import numpy as np

from utils.config import load_config, save_config

In [5]:
config_path = "configs/baseline_engagements.json"

config = load_config(config_path)
config

{'name': 'baseline_engagements',
 'seed': 42,
 'datapath': '../problem_merged_data.csv',
 'pre_process': {'processed_data_path': 'processed_data/baseline_engage_0.8.pkl',
  'num_input': ['bid', 'budget'],
  'cat_input': None,
  'target_output': 'engagements',
  'train_split': 0.8},
 'train': {'cat_emb_dim': 4,
  'learning_rate': 0.001,
  'decay_steps': 20,
  'decay_rate': 0.98,
  'dense_dims': [1024, 1024, 1024, 1024, 1024, 1024, 16],
  '_drop_out_rate': 0.2,
  'use_test_as_valid': True,
  'valid_split': 0.1,
  'batch_size': 128,
  'epochs': 100},
 'cpe_min': {'engage_model_path': 'experiments/baseline_engagements/model/model.keras',
  'media_spend_model_path': 'experiments/baseline_media_spend/model/model.keras',
  'engage_data': 'processed_data/baseline_engage_0.8.pkl',
  'spend_data': 'processed_data/baseline_spend_0.8.pkl',
  'bid_range': [0.1, 1.1],
  'budget_range': [5.9, 680],
  'n_grid': 20,
  'sample_idx': 170,
  'max_media_spend': None}}

In [15]:
#TRAIN_SPLIT = [0.7, 0.8, 0.9]
#CAT_EMB_DIM = [2, 4, 8, 16, 32, 64, 128]

TRAIN_SPLIT = [0.8]
CAT_EMB_DIM = [4, 16, 64] #[2, 4, 8, 16, 32, 64, 128]


def select_random(_list):
    np.random.shuffle(_list)
    return _list[0]

In [16]:
NUM_MID_DENSE = [6, 7] 
DENSE_MID_DIM = [1024, 2048]
END_DIM = [16, 32, 64, 128] #[2, 4, 8, 16]

def gen_random_arch():
    num_mid_dense = select_random(NUM_MID_DENSE)
    dense_mid_dim = select_random(DENSE_MID_DIM)
    end_dim = select_random(END_DIM)
    
    dense_dims = [dense_mid_dim for _ in range(num_mid_dense)]
    dense_dims.append(end_dim)
    return dense_dims

In [17]:
LR = [0.001] # [0.01, 0.001, 0.0001]
DROP_OUT = [0.2, 0.3] #[0.1, 0.2, 0.3, 0.4, 0.5]

def get_random_in_range(MAX, MIN):
    return (MAX-MIN)*np.random.rand() + MIN

In [18]:
train_split = select_random(TRAIN_SPLIT)
cat_emb_dim = select_random(CAT_EMB_DIM)

dense_dims = gen_random_arch()

learning_rate = np.round(10**get_random_in_range(-2, -4), 8)
drop_out = np.round(get_random_in_range(0.5, 0.1), 8)

print("Random Examples:", train_split, cat_emb_dim, dense_dims, learning_rate, drop_out)

Random Examples: 0.8 16 [2048, 2048, 2048, 2048, 2048, 2048, 2048, 32] 0.00080991 0.26784373


#### Random Config Engagement

In [19]:
config_path = "configs/baseline_engagements.json"
config = load_config(config_path)

In [20]:
for _n_rand in range(20):
    new_config = config.copy()
    
    train_split = select_random(TRAIN_SPLIT)
    
    cat_emb_dim = select_random(CAT_EMB_DIM)
    dense_dims = gen_random_arch()

    learning_rate = np.round(10**get_random_in_range(-2, -4), 8)
    drop_out = np.round(get_random_in_range(0.5, 0.1), 8)
    
    
    new_config["name"] = config["name"]+f"_{_n_rand}"
    new_config["pre_process"]["train_split"] = train_split
    new_config["pre_process"]["processed_data_path"] = f'processed_data/baseline_engagement_{train_split}.pkl'
    
    new_config["train"]["cat_emb_dim"] = cat_emb_dim
    new_config["train"]["dense_dims"] = dense_dims
    new_config["train"]["learning_rate"] = learning_rate
    new_config["train"]["_drop_out_rate"] = drop_out
    
    path_out = f"configs/random_search/{new_config['name']}.json"
    save_config(path_out, new_config)

#### Random Config Media Spend

In [21]:
config_path = "configs/baseline_media_spend.json"
config = load_config(config_path)

In [22]:
for _n_rand in range(20):
    new_config = config.copy()
    
    train_split = select_random(TRAIN_SPLIT)
    
    cat_emb_dim = select_random(CAT_EMB_DIM)
    dense_dims = gen_random_arch()

    learning_rate = np.round(10**get_random_in_range(-2, -4), 8)
    drop_out = np.round(get_random_in_range(0.5, 0.1), 8)
    
    
    new_config["name"] = config["name"]+f"_{_n_rand}"
    new_config["pre_process"]["train_split"] = train_split
    new_config["pre_process"]["processed_data_path"] = f'processed_data/baseline_media_spend_{train_split}.pkl'
    
    new_config["train"]["cat_emb_dim"] = cat_emb_dim
    new_config["train"]["dense_dims"] = dense_dims
    new_config["train"]["learning_rate"] = learning_rate
    new_config["train"]["_drop_out_rate"] = drop_out
    
    path_out = f"configs/random_search/{new_config['name']}.json"
    save_config(path_out, new_config)

#### Make Scripts

In [23]:
import glob

In [24]:
list_random_configs = glob.glob("configs/random_search/*json")
list_random_configs.sort(key=os.path.getmtime)
len(list_random_configs)

40

In [25]:
cmds = [f"bash main.sh {elem}\n" for elem in list_random_configs]
cmds[:3]

['bash main.sh configs/random_search/baseline_engagements_0.json\n',
 'bash main.sh configs/random_search/baseline_engagements_1.json\n',
 'bash main.sh configs/random_search/baseline_engagements_2.json\n']

In [26]:
with open("run_random_search_baseline.sh", "w") as fhandle:
    for cmd in cmds:
        fhandle.write(cmd)

#### Baseline Search

In [6]:
config_path = "configs/baseline_engagements.json"
config = load_config(config_path)

In [7]:
idx=0

for num_mid_dense in [6, 7]:
    for dense_mid_dim in [1024, 2048]:
        for cat_emb_dim in [4, 8, 16]:
            for end_dim in [8, 16, 32, 64]:
                idx+=1
                
                new_config = config.copy()
                new_config["name"] = config["name"]+f"_{idx}"
                
                dense_dims = [dense_mid_dim for _ in range(num_mid_dense)]
                dense_dims.append(end_dim)

                new_config["train"]["cat_emb_dim"] = cat_emb_dim
                new_config["train"]["dense_dims"] = dense_dims

                path_out = f"configs/baseline_search/{new_config['name']}.json"
                save_config(path_out, new_config)

In [8]:
config_path = "configs/baseline_media_spend.json"
config = load_config(config_path)

In [9]:
idx=0

for num_mid_dense in [6, 7]:
    for dense_mid_dim in [1024, 2048]:
        for cat_emb_dim in [4, 8, 16]:
            for end_dim in [8, 16, 32, 64]:
                idx+=1
                
                new_config = config.copy()
                new_config["name"] = config["name"]+f"_{idx}"
                
                dense_dims = [dense_mid_dim for _ in range(num_mid_dense)]
                dense_dims.append(end_dim)

                new_config["train"]["cat_emb_dim"] = cat_emb_dim
                new_config["train"]["dense_dims"] = dense_dims

                path_out = f"configs/baseline_search/{new_config['name']}.json"
                save_config(path_out, new_config)

In [11]:
# Make Run (!)
list_random_configs = glob.glob("configs/baseline_search/*json")
list_random_configs.sort(key=os.path.getmtime)
len(list_random_configs)

96

In [12]:
cmds = [f"bash main.sh {elem}\n" for elem in list_random_configs]
cmds[:3]

['bash main.sh configs/baseline_search/baseline_engagements_1.json\n',
 'bash main.sh configs/baseline_search/baseline_engagements_2.json\n',
 'bash main.sh configs/baseline_search/baseline_engagements_3.json\n']

In [13]:
with open("run_search_baseline.sh", "w") as fhandle:
    for cmd in cmds:
        fhandle.write(cmd)